In [1]:
using Printf, CSV, DataFrames, Statistics, Glob, XLSX, Dates

In [2]:
directory = joinpath(homedir(), "research", "ERCOT_LOAD_DATA", "raw")

"/home/emco4286/research/ERCOT_LOAD_DATA/raw"

In [3]:
x = Dict()
i = 1

directory = joinpath(homedir(), "research", "ERCOT_LOAD_DATA", "raw")
files = glob("*.csv", directory)
# types_dict = Dict("Hour_End" => DateTime, "HourEnding" => DateTime, "Hour Ending" => DateTime)
date_format_1 = dateformat"mm/dd/yyyy HH:MM"
date_format_2 = dateformat"mm/dd/yyyy HH:MM D\S\T"

for f in files

    df = DataFrame(CSV.File(f)) #, types=types_dict, validate=false))

    if "NCENT" in names(df)
        rename!(df, :NCENT => :NORTH_C)
    end

    if "HourEnding" in names(df)
        rename!(df, :HourEnding => :Hour_End)
    end

    if "Hour Ending" in names(df)
        rename!(df, "Hour Ending" => :Hour_End)
    end

    df = df[!, ["Hour_End", "COAST", "NORTH_C"]]

    
    mylist = Vector{DateTime}()
    for dt in df[!, :Hour_End]

        try
            mydt = DateTime(dt, date_format_1)
            push!(mylist, mydt)
        catch e
            mydt = DateTime(dt, date_format_2) - Hour(1)
            push!(mylist, mydt)
            
        end
    end

    df[!, "Hour_End"] = mylist

    x[i] = df
    i +=1
end

merged = reduce(vcat, values(x))
sort!(merged, :Hour_End)


Row,Hour_End,COAST,NORTH_C
,DateTime,Any,Any
1,2013-01-01T01:00:00,7607.19,10370.3
2,2013-01-01T02:00:00,7389.27,10153.9
3,2013-01-01T03:00:00,7179.75,9989.24
4,2013-01-01T04:00:00,7039.69,9947.86
5,2013-01-01T05:00:00,6991.41,10097.4
6,2013-01-01T06:00:00,7050.73,10472.1
7,2013-01-01T07:00:00,7177.79,10891.2
8,2013-01-01T08:00:00,7223.21,11190.2
9,2013-01-01T09:00:00,7314.25,11627.1


In [ ]:
s = merged[!, "COAST"]
mylist = Vector{Float64}()
i = 0

for v in s
    if v isa String15
        v2 = replace(v, ','=>"", " " =>"")
        v3 = parse.(Float64, v2)
    elseif v isa Missing
        v3 = NaN64
    elseif typeof(v) != Float64
        v3 = parse.(Float64, v)
    else
        v3 = v
    end
    push!(mylist, v3)
    i += 1
end

merged[!, "COAST"] = mylist

In [28]:
s = merged[!, "NORTH_C"]
mylist = Vector{Float64}()
i = 0

for v in s
    if v isa String15
        v2 = replace(v, ','=>"", " " =>"")
        v3 = parse.(Float64, v2)
    elseif v isa Missing
        v3 = NaN64
    elseif typeof(v) != Float64
        v3 = parse.(Float64, v)
    else
        v3 = v
    end
    push!(mylist, v3)
    i += 1
end

merged[!, "NORTH_C"] = mylist

96408-element Vector{Float64}:
 10370.32292
 10153.94281
  9989.243532
  9947.855512
 10097.44423
 10472.12636
 10891.2179
 11190.20403
 11627.12738
 12310.00187
     ⋮
 11209.26
 11377.89
 12082.31
 12630.86
 12423.9
 12226.08
 12039.3
 11844.79
 11829.84

In [29]:
fname = "ercot_hourly_load_2013_2023.csv"
path = joinpath(homedir(), "research", "ERCOT_LOAD_DATA", "processed", fname)
CSV.write(path, merged)

"/home/emco4286/research/ERCOT_LOAD_DATA/processed/ercot_hourly_load_2013_2023.csv"